In [31]:
import pandas as pd

In [32]:
df_original = pd.read_csv("../train_data.csv")
n_original = df_original.shape[0]
df_submit = pd.read_csv("../sample_submission.csv")
df = pd.concat([df_original, df_submit], axis=0).reset_index(drop=True)

# Features from GRU
df = pd.read_csv("../GRU_features_predict_only.csv").merge(
    df,
    on='id'
)


In [33]:
def siRNA_feat_builder(s: pd.Series, anti: bool = False):
    name = "anti" if anti else "sense"
    df = s.to_frame()
    df[f"feat_siRNA_{name}_seq_len"] = s.str.len()
    for pos in [0, -1]:
        for c in list("AUGC"):
            df[f"feat_siRNA_{name}_seq_{c}_{'front' if pos == 0 else 'back'}"] = (
                s.str[pos] == c
            )
    df[f"feat_siRNA_{name}_seq_pattern_1"] = s.str.startswith("AA") & s.str.endswith(
        "UU"
    )
    df[f"feat_siRNA_{name}_seq_pattern_2"] = s.str.startswith("GA") & s.str.endswith(
        "UU"
    )
    df[f"feat_siRNA_{name}_seq_pattern_3"] = s.str.startswith("CA") & s.str.endswith(
        "UU"
    )
    df[f"feat_siRNA_{name}_seq_pattern_4"] = s.str.startswith("UA") & s.str.endswith(
        "UU"
    )
    df[f"feat_siRNA_{name}_seq_pattern_5"] = s.str.startswith("UU") & s.str.endswith(
        "AA"
    )
    df[f"feat_siRNA_{name}_seq_pattern_6"] = s.str.startswith("UU") & s.str.endswith(
        "GA"
    )
    df[f"feat_siRNA_{name}_seq_pattern_7"] = s.str.startswith("UU") & s.str.endswith(
        "CA"
    )
    df[f"feat_siRNA_{name}_seq_pattern_8"] = s.str.startswith("UU") & s.str.endswith(
        "UA"
    )
    df[f"feat_siRNA_{name}_seq_pattern_9"] = s.str[1] == "A"
    df[f"feat_siRNA_{name}_seq_pattern_10"] = s.str[-2] == "A"
    df[f"feat_siRNA_{name}_seq_pattern_GC_ratio_0"] = (
        s.str.count("G") + s.str.count("C")
    ) / s.str.len()

    df[f"feat_siRNA_{name}_len_range"] = (s.str.len() >= 21) & (s.str.len() <= 25)

    GC_ratio_1 = (s.str.count("G") + s.str.count("C")) / s.str.len()
    df[f"feat_siRNA_{name}_GC_ratio_1"] = (GC_ratio_1 >= 0.31) & (GC_ratio_1 <= 0.58)

    GC_ratio_2 = (s.str[1:7].str.count("G") + s.str[1:7].str.count("C")) / s.str[1:7].str.len()
    df[f"feat_siRNA_{name}_GC_ratio_2"] = (GC_ratio_2 == 0.19)

    GC_ratio_3 = (s.str[7:18].str.count("G") + s.str[7:18].str.count("C")) / s.str[7:18].str.len()
    df[f"feat_siRNA_{name}_GC_ratio_3"] = (GC_ratio_3 == 0.52)

    return df.iloc[:, 1:]

In [34]:
df_publication_id = pd.get_dummies(df.publication_id)
df_publication_id.columns = [
    f"feat_publication_id_{c}" for c in df_publication_id.columns
]
df_gene_target_symbol_name = pd.get_dummies(df.gene_target_symbol_name)
df_gene_target_symbol_name.columns = [
    f"feat_gene_target_symbol_name_{c}" for c in df_gene_target_symbol_name.columns
]
df_gene_target_ncbi_id = pd.get_dummies(df.gene_target_ncbi_id)
df_gene_target_ncbi_id.columns = [
    f"feat_gene_target_ncbi_id_{c}" for c in df_gene_target_ncbi_id.columns
]
df_gene_target_species = pd.get_dummies(df.gene_target_species)
df_gene_target_species.columns = [
    f"feat_gene_target_species_{c}" for c in df_gene_target_species.columns
]
siRNA_duplex_id_values = df.siRNA_duplex_id.str.split("-|\.").str[1].astype("int")
siRNA_duplex_id_values = (siRNA_duplex_id_values - siRNA_duplex_id_values.min()) / (
    siRNA_duplex_id_values.max() - siRNA_duplex_id_values.min()
)
df_siRNA_duplex_id = pd.DataFrame(siRNA_duplex_id_values)
df_cell_line_donor = pd.get_dummies(df.cell_line_donor)
df_cell_line_donor.columns = [
    f"feat_cell_line_donor_{c}" for c in df_cell_line_donor.columns
]
df_cell_line_donor["feat_cell_line_donor_hepatocytes"] = (
    (df.cell_line_donor.str.contains("Hepatocytes")).fillna(False).astype("int")
)
df_cell_line_donor["feat_cell_line_donor_cells"] = (
    df.cell_line_donor.str.contains("Cells").fillna(False).astype("int")
)
df_siRNA_concentration = df.siRNA_concentration.to_frame()
df_Transfection_method = pd.get_dummies(df.Transfection_method)
df_Transfection_method.columns = [
    f"feat_Transfection_method_{c}" for c in df_Transfection_method.columns
]
df_Duration_after_transfection_h = pd.get_dummies(df.Duration_after_transfection_h)
df_Duration_after_transfection_h.columns = [
    f"feat_Duration_after_transfection_h_{c}"
    for c in df_Duration_after_transfection_h.columns
]

df_GRU_pred = df[['GRU_predict']]

'''
# More GRU realted features for experimentation
df_GRU_feature = df[ df.columns[['GRU_feature_' in c for c in df.columns]] ]
important_GRU_features = [
    'GRU_feature_11','GRU_feature_18','GRU_feature_523','GRU_feature_679','GRU_feature_689'
]
df_GRU_feature = df_GRU_feature[important_GRU_features]
'''

feats = pd.concat(
    [
        df_publication_id,
        df_gene_target_symbol_name,
        df_gene_target_ncbi_id,
        df_gene_target_species,
        df_siRNA_duplex_id,
        df_cell_line_donor,
        df_siRNA_concentration,
        df_Transfection_method,
        df_Duration_after_transfection_h,
        siRNA_feat_builder(df.siRNA_sense_seq, False),
        siRNA_feat_builder(df.siRNA_antisense_seq, True),
        df_GRU_pred,
        # df_GRU_feature,
        df.iloc[:, -1].to_frame(),
    ],
    axis=1,
)

In [35]:
feats.shape

(30656, 212)

In [36]:
import lightgbm as lgb
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_error
import numpy as np
from xgboost import XGBRegressor, callback as xgb_callback
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import StackingRegressor
from sklearn.metrics import mean_absolute_error

features = feats.iloc[:n_original, :-1]
targets = feats.iloc[:n_original, -1]
X_train, X_test, y_train, y_test = train_test_split(
    features,
    targets,
    test_size=0.2,
    random_state=42,
)

In [37]:
from sklearn.metrics import mean_absolute_error, make_scorer

# calculate_validation_score for GridSearchCV
def calculate_validation_score(y_true, y_pred, threshold=30):
    # y_pred = preds
    # y_true = data.get_label()
    mae = np.mean(np.abs(y_true - y_pred))
    # if mae < 0: mae = 0
    # elif mae >100: mae = 100

    y_true_binary = ((y_true <= threshold) & (y_true >= 0)).astype(int)
    y_pred_binary = ((y_pred <= threshold) & (y_pred >= 0)).astype(int)

    mask = (y_pred >= 0) & (y_pred <= threshold)
    range_mae = (
        mean_absolute_error(y_true[mask], y_pred[mask]) if np.sum(mask) > 0 else 100
    )
    # if range_mae < 0: range_mae = 0
    # elif range_mae >100: range_mae = 100

    # precision = precision_score(y_true_binary, y_pred_binary, average="binary")
    # recall = recall_score(y_true_binary, y_pred_binary, average="binary")

    if np.sum(y_pred_binary) > 0:
        precision = (np.array(y_pred_binary) & y_true_binary).sum()/np.sum(y_pred_binary)
    else:
        precision = 0
    if np.sum(y_true_binary) > 0:
        recall = (np.array(y_pred_binary) & y_true_binary).sum()/np.sum(y_true_binary)
    else:
        recall = 0

    if precision + recall == 0:
        f1 = 0
    else:
        f1 = 2 * precision * recall / (precision + recall)
    score = (1 - mae / 100) * 0.5 + (1 - range_mae / 100) * f1 * 0.5
    return score

custom_scorer = make_scorer(calculate_validation_score, greater_is_better=True)


In [38]:
# calculate_metrics for lightgbm training
def calculate_validation_score_for_training(preds, data, threshold=30):
    y_pred = preds
    y_true = data.get_label()
    mae = np.mean(np.abs(y_true - y_pred))
    # if mae < 0: mae = 0
    # elif mae >100: mae = 100

    y_true_binary = ((y_true <= threshold) & (y_true >= 0)).astype(int)
    y_pred_binary = ((y_pred <= threshold) & (y_pred >= 0)).astype(int)

    mask = (y_pred >= 0) & (y_pred <= threshold)
    range_mae = (
        mean_absolute_error(y_true[mask], y_pred[mask]) if np.sum(mask) > 0 else 100
    )
    # if range_mae < 0: range_mae = 0
    # elif range_mae >100: range_mae = 100

    # precision = precision_score(y_true_binary, y_pred_binary, average="binary")
    # recall = recall_score(y_true_binary, y_pred_binary, average="binary")

    if np.sum(y_pred_binary) > 0:
        precision = (np.array(y_pred_binary) & y_true_binary).sum()/np.sum(y_pred_binary)
    else:
        precision = 0
    if np.sum(y_true_binary) > 0:
        recall = (np.array(y_pred_binary) & y_true_binary).sum()/np.sum(y_true_binary)
    else:
        recall = 0

    if precision + recall == 0:
        f1 = 0
    else:
        f1 = 2 * precision * recall / (precision + recall)
    score = (1 - mae / 100) * 0.5 + (1 - range_mae / 100) * f1 * 0.5
    return "custom_score", score, True


In [39]:
#######
# For the training data, double the size of the observations with y < 30:
def Get_sample_weight(y):
    weights = list(map(lambda x: 2 if (x<=30 and x>=0) else 1, y))
    return np.array(weights)
#######

train_data = lgb.Dataset(X_train, label=y_train, weight=Get_sample_weight(y_train))
test_data = lgb.Dataset(X_test, label=y_test, reference=train_data, weight=Get_sample_weight(y_test))

In [40]:

# change loss_method to "custom" if you want to use same method recommended by the competition 
loss_method = "RMSE"
using_stacking = True

# 定义一个回调函数来打印验证集的结果
def print_validation_result(env):
    result = env.evaluation_result_list[-1]
    print(f"[{env.iteration}] {result[1]}'s {result[0]}: {result[2]}")

'''
# Grid Search for Hyperparameter Tuning (Optional)
param_grid = {
    'max_depth': [7, 9, 11],
    'learning_rate': [0.01, 0.02],
    'num_leaves': [31, 63, 127],
    'feature_fraction': [0.8, 0.9],
    'bagging_fraction': [0.8, 0.9],
    'bagging_freq': [0, 5, 10],
    'n_estimators': [8700, 15000, 20000],
    'min_child_samples': [20, 30, 50],
}

if loss_method == "RMSE"
    gbm = lgb.LGBMRegressor(boosting_type='gbdt', objective='regression', metric='rmse', n_estimators=15000)
    grid = GridSearchCV(gbm, param_grid, cv=3, scoring='neg_root_mean_squared_error', verbose=1)
else:
    gbm = lgb.LGBMRegressor(boosting_type='gbdt', objective='regression')
    grid = GridSearchCV(gbm, param_grid, cv=3, scoring=custom_scorer, verbose=1)

grid.fit(features, targets)

print(f'Best parameters found by grid search are: {grid.best_params_}')
print(f'Best estimator found by grid search are: {grid.best_estimator_}')

# Train with best parameters
best_params = grid.best_params_
best_estimator = grid.best_estimator_
'''

'\n# Grid Search for Hyperparameter Tuning (Optional)\nparam_grid = {\n    \'max_depth\': [7, 9, 11],\n    \'learning_rate\': [0.01, 0.02],\n    \'num_leaves\': [31, 63, 127],\n    \'feature_fraction\': [0.8, 0.9],\n    \'bagging_fraction\': [0.8, 0.9],\n    \'bagging_freq\': [0, 5, 10],\n    \'n_estimators\': [8700, 15000, 20000],\n    \'min_child_samples\': [20, 30, 50],\n}\n\nif loss_method == "RMSE"\n    gbm = lgb.LGBMRegressor(boosting_type=\'gbdt\', objective=\'regression\', metric=\'rmse\', n_estimators=15000)\n    grid = GridSearchCV(gbm, param_grid, cv=3, scoring=\'neg_root_mean_squared_error\', verbose=1)\nelse:\n    gbm = lgb.LGBMRegressor(boosting_type=\'gbdt\', objective=\'regression\')\n    grid = GridSearchCV(gbm, param_grid, cv=3, scoring=custom_scorer, verbose=1)\n\ngrid.fit(features, targets)\n\nprint(f\'Best parameters found by grid search are: {grid.best_params_}\')\nprint(f\'Best estimator found by grid search are: {grid.best_estimator_}\')\n\n# Train with best par

In [41]:

# xgb_lr_array
xgb_lr = []
# lgb_lr_array
lgb_lr = []

def adaptive_learning_rate(decay_rate=0.8, patience=50):
    best_score = float("inf")  # initialize with positive infinity since a lower score is better.
    wait = 0

    def callback(env):
        nonlocal best_score, wait
        # Find the score in evals_log[-1][2]
        current_score = env.evaluation_result_list[-1][2]
        current_lr =  env.model.params.get('learning_rate')

        # TODO: save learning rate 
        # lgb_lr.append(current_lr)

        if current_score < best_score: 
            best_score = current_score
            # wait = 0 # the score needs to have no continuous improvement.
        else:
            wait += 1

        if wait >= patience:
            new_lr = float(current_lr) * decay_rate
            wait = 0
            env.model.params['learning_rate'] = new_lr
            print(f"Learning rate adjusted to {env.model.params.get('learning_rate')}")

    return callback

def custom_rmse(y_true, y_pred):
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    return 'custom_rmse', rmse, False

def custom_rmse_for_xgb(y_true, y_pred):
    # Calculate RMSE
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    # Return the metric name, value, and whether higher values are better (False for RMSE)
    return 'custom_rmse', rmse


class AdaptiveLearningRateCallback(xgb_callback.TrainingCallback):
    def __init__(self, decay_rate=0.8, patience=50, initial_lr=0.1):
        self.decay_rate = decay_rate
        self.patience = patience
        self.best_score = float('inf')
        self.wait = 0
        self.current_lr = initial_lr

    def after_iteration(self, model, epoch, evals_log):
        if 'validation_0' in evals_log and 'rmse' in evals_log['validation_0']:
            current_score = evals_log['validation_0']['rmse'][-1]
            if current_score < self.best_score:
                self.best_score = current_score
                self.wait = 0
            else:
                self.wait += 1

            # TODO: save learning rate 
            # xgb_lr.append(self.current_lr)

            if self.wait >= self.patience:
                self.current_lr *= self.decay_rate
                model.set_param('learning_rate', self.current_lr)
                self.wait = 0
                print(f"Learning rate adjusted to {self.current_lr}")
        else:
            # Debugging: print available keys
            print("Available keys in evals_log:", evals_log.keys())

        return False  # Continue training


In [ ]:
# saved best parameters to improve testing efficiency
best_estimator = {
    "n_estimators": 15000,
}

# Custom callback to apply adaptive learning rate for LightGbm
adaptive_lr = adaptive_learning_rate(decay_rate=0.9, patience=1000)

# Adaptive learning rate for XGBoost using a learning rate scheduler and training data
adaptive_lr_callback = AdaptiveLearningRateCallback(decay_rate=0.8, patience=50)

# LightGBM model with custom learning rate callback
lgb_reg = lgb.LGBMRegressor(
    boosting_type='gbdt',
    objective='regression',
    min_child_samples=20,
    num_leaves=127,
    max_depth=9,
    learning_rate=0.02,  # Initial learning rate
    n_estimators=best_estimator["n_estimators"]
)

# Fitting LightGBM model with custom callback and training data
lgb_reg.fit(X_train, y_train,
            sample_weight=Get_sample_weight(y_train),
            callbacks=[
                adaptive_lr,
                lgb.early_stopping(stopping_rounds=int(best_estimator["n_estimators"] * 0.1)),
                print_validation_result
            ],
            eval_metric=custom_rmse,
            eval_set=[(X_test, y_test)])

# XGBoost model with learning rate scheduler
xgb_reg = XGBRegressor(
    objective='reg:squarederror',
    n_estimators=best_estimator["n_estimators"],
    callbacks=[adaptive_lr_callback],
    eval_metric='rmse',
    learning_rate=0.1,  # Initial learning rate
)

xgb_reg.fit(X_train, y_train,
            sample_weight=Get_sample_weight(y_train),
            eval_set=[(X_test, y_test)])

# Create meta-features from the predictions of the base estimators
meta_features_train = np.column_stack([
    lgb_reg.predict(X_train),
    xgb_reg.predict(X_train)
])

meta_features_test = np.column_stack([
    lgb_reg.predict(X_test),
    xgb_reg.predict(X_test)
])

# Define and fit the final estimator
stacking_estimator = LinearRegression()
stacking_estimator.fit(meta_features_train, y_train)

if loss_method == "RMSE":
    best_params = {
        "boosting_type": "gbdt",
        "objective": "regression",
         "metric": "root_mean_squared_error",
        "bagging_fraction": 0.8,
        "bagging_freq": 5,
        "feature_fraction": 0.9,
        "learning_rate": 0.02,
        "max_depth": 9,
        "num_leaves": 127,
        "min_child_samples": 20
    }

    if using_stacking:
        stacking_estimator.fit(meta_features_train, y_train)
    else:
        best_gbm = lgb.train(
            best_params,
            train_data,
            num_boost_round=best_estimator["n_estimators"],
            valid_sets=[train_data, test_data],
            callbacks=[
                lgb.early_stopping(stopping_rounds=int(best_estimator["n_estimators"] * 0.1)),
                print_validation_result
            ],
        )
else:
    best_params = {
        "boosting_type": "gbdt",
        "objective": "regression",
        "metric": "None",
        "bagging_fraction": 0.8,
        "bagging_freq": 5,
        "feature_fraction": 0.9,
        "learning_rate": 0.02,
        "max_depth": 9,
        "num_leaves": 127,
        "min_child_samples": 20
    }
    
    best_gbm = lgb.train(
        best_params,
        train_data,
        num_boost_round=best_estimator["n_estimators"],
        feval=calculate_validation_score_for_training,
        valid_sets=[train_data, test_data],
        callbacks=[
            lgb.early_stopping(stopping_rounds=int(best_estimator["n_estimators"] * 0.1)),
            print_validation_result
        ],
    )

# Final Evaluation
if using_stacking:
    y_pred_eval = stacking_estimator.predict(meta_features_test)
else:
    y_pred_eval = best_gbm.predict(X_test, num_iteration=best_gbm.best_iteration)

if loss_method == "RMSE":
    rmse_best = np.sqrt(mean_squared_error(y_test, y_pred_eval))
    print(f'Validation RMSE after tuning: {rmse_best}')
else:
    score_best = calculate_validation_score(y_test, y_pred_eval)
    print(f'Validation score after tuning: {score_best}')

# best score is 0.8288


In [28]:
# Finial train and predict

if using_stacking:
    best_model = stacking_estimator
else:
    all_data = lgb.Dataset(features, label=targets, weight=Get_sample_weight(targets))
    best_model = lgb.train(
        best_params,
        all_data,
        num_boost_round=best_estimator["n_estimators"]
    )


In [29]:
eval_data = feats.iloc[n_original:, :-1]

if using_stacking:
    meta_features_eval = np.column_stack([
        lgb_reg.predict(eval_data),
        xgb_reg.predict(eval_data)
    ])
    eval_data = meta_features_eval

y_pred_best = best_model.predict(eval_data)

In [30]:
df_submit["mRNA_remaining_pct"] = y_pred_best
df_submit.to_csv("../submission.csv", index=False)